In [108]:
from dateutil import rrule
from datetime import datetime
from datetime import timedelta
import string
import math
import random
# ## rrule
characters = string.ascii_letters + string.digits
def get_random_string(length: int) -> str:
    return ''.join(random.choice(characters) for i in range(length))

def get_random_date_from(start_date):
    return start_date + timedelta(days=random.randint(0,1300))
    

# function that returns random timestamps


def get_random_date():
    minutes_in_year = 365 * 24 * 60
    return datetime.now() + timedelta(minutes=random.randint(200, minutes_in_year*10))
def get_random_unique_days_of_week_list():
    days_of_weeks = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    days = []
    for i in days_of_weeks:
        if random.randint(0,200)%2 == 0:
            days.append(i)
    return days
def get_random_end(start_date):
    if random.randint(0,200)%2 == 0:
        return "Never"
    else:
        if random.randint(1, 100)%2 == 0:
            return f"{random.randint(0,100)} Occurrences"
        else :
            return get_random_date_from(start_date)
def get_random_recurrence(original_date):
    repeat_every = random.randint(1, 100)
    repeat_by = random.choice(['Day', 'Week', 'Month', 'Year'])
    repeat_on = get_random_unique_days_of_week_list()
    ends_on = get_random_end(original_date)
    return {
        "repeatsEvery": repeat_every,
        "repeatsBy": repeat_by,
        "repeatsOn": repeat_on,
        "endsOn": ends_on,
    }
def get_rrule_string(original_date:datetime,interval_type,ceil_count=100) -> str:
    date_map = {
        "Monday":"MO",
        "Tuesday":"TU",
        "Wednesday":"WE",
        "Thursday":"TH",
        "Friday":"FR",
        "Saturday":"SA",
        "Sunday":"SU",
    }
    if interval_type == "Daily":
       return f"RRULE:FREQ=DAILY;COUNT={ceil_count}"
    elif interval_type == "Weekly On Day":
        current_week = original_date.isocalendar()[1]
        day = original_date.strftime("%A")
        rrule_day = date_map[day]
        return f"RRULE:FREQ=WEEKLY;BYDAY={rrule_day};COUNT={ceil_count}"
    elif interval_type == "Monthly On Current Day Of Current Week":
        day = date_map[original_date.strftime("%A")]
        week_number=get_week_of_month(original_date)
        return f"RRULE:FREQ=MONTHLY;BYDAY={week_number}{day};COUNT={ceil_count}"
    elif interval_type == "Yearly On Current Date Of Current Month":
        day = int(original_date.strftime("%d"))
        month = int(original_date.strftime("%m"))
        return f"RRULE:FREQ=YEARLY;BYMONTH={month};BYMONTHDAY={day};COUNT={ceil_count}"
    elif interval_type == "Every Weekday":
        return f"RRULE:FREQ=WEEKLY;BYDAY=MO,TU,WE,TH,FR;COUNT={ceil_count}"
    elif interval_type == "Custom":
        data = get_random_recurrence(original_date)
        ends_on = data["endsOn"]
        repeats_by = data["repeatsBy"]
        repeats_on = data["repeatsOn"]
        repeasts_every = data["repeatsEvery"]
    else:
        pass
        
def get_week_of_month(date):
   first_day = date.replace(day=1)

   day_of_month = date.day

   if(first_day.weekday() == 6):
       adjusted_dom = (1 + first_day.weekday()) / 7
   else:
       adjusted_dom = day_of_month + first_day.weekday()

   return int(math.ceil(adjusted_dom/7.0))


In [109]:
from faker import Faker
fk = Faker()
def generate_meeting():
    rcDetial = random.choice(["Daily", "Weekly On Day", "Monthly On Current Day Of Current Week",
                              "Yearly On Current Date Of Current Month", "Every Weekday"])
    date = get_random_date()
    rrule_string = get_rrule_string(date, rcDetial, ceil_count=random.randint(1, 100))
    return {
        "hostingSubDivisionId": get_random_string(10),
        "isParent": True,
        "rrule": rrule_string,
        "recurrenceDetail": rcDetial,
        "hasEndDate": random.choice([True, False]),
        "endDate": get_random_date_from(date).isoformat(),
        "meetingTime": date.strftime("%H:%M %p"),
        "meetingType": fk.sentence(),
    }


meeting_list = [generate_meeting() for _ in range(100)]
meeting_list[0:4]


[{'hostingSubDivisionId': 'O6RZZVvubi',
  'isParent': True,
  'rrule': 'RRULE:FREQ=MONTHLY;BYDAY=2MO;COUNT=76',
  'recurrenceDetail': 'Monthly On Current Day Of Current Week',
  'hasEndDate': True,
  'endDate': '2028-01-19T02:49:04.337109',
  'meetingTime': '02:49 AM',
  'meetingType': 'Cost huge key culture reflect.'},
 {'hostingSubDivisionId': 'tZYrh18AG4',
  'isParent': True,
  'rrule': 'RRULE:FREQ=WEEKLY;BYDAY=MO,TU,WE,TH,FR;COUNT=46',
  'recurrenceDetail': 'Every Weekday',
  'hasEndDate': False,
  'endDate': '2032-08-10T09:15:04.337182',
  'meetingTime': '09:15 AM',
  'meetingType': 'I tell offer team world their sort.'},
 {'hostingSubDivisionId': 'HETUaLtaKS',
  'isParent': True,
  'rrule': 'RRULE:FREQ=WEEKLY;BYDAY=MO,TU,WE,TH,FR;COUNT=81',
  'recurrenceDetail': 'Every Weekday',
  'hasEndDate': True,
  'endDate': '2026-11-02T02:03:04.337203',
  'meetingTime': '02:03 AM',
  'meetingType': 'Blood between station table.'},
 {'hostingSubDivisionId': '9h4RNxkPcd',
  'isParent': True,


In [110]:
def make_list_from_rrule(data,start_date=None,end_date=None):
    rrule_str = data["rrule"]
    rrule_dt_list = []
    final_list = []
    if start_date is None:
        rrule_dt_list = list(rrule.rrulestr(rrule_str))                
    else:
        rrule_dt_list = list(rrule.rrulestr(rrule_str,dtstart=start_date,until=end_date))
    for i in rrule_dt_list:
        final_list.append(
            {
                **data,
                "meetingDate": i.isoformat(),
            }
        )
    return final_list

        

In [111]:
from concurrent.futures import ThreadPoolExecutor,as_completed
import pprint
def get_final_unsorted_list():
    unsorted_list = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_data = {executor.submit(make_list_from_rrule,data):data for data in meeting_list}
        for future in as_completed(future_to_data):
            data = future_to_data[future]
            final_list = future.result()
            unsorted_list.extend(final_list)
    return unsorted_list
def sort_meeting_by_date(unsorted_arr):
    sorted_list = sorted(unsorted_arr, key=lambda k: k['meetingDate'])
    return sorted_list
un_data = get_final_unsorted_list()
sorted_data = sort_meeting_by_date(un_data)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(len(sorted_data))
import json
#save to file
with open("meeting_data.json","w") as f:
    json.dump(sorted_data,f)


5175


In [112]:

import re
import pytz
current_date = f'{datetime.now().isoformat().replace("-", "").replace(":", "")}Z'
string_d = 'RRULE:FREQ=WEEKLY;BYDAY=MO,TU,WE,TH,FR;COUNT=79'
final_string = re.sub(r'COUNT=\d*', f'UNTIL={current_date}', string_d)
list(rrule.rrulestr(final_string))
def make_list_from_rrule(data):
    dt = data[0]
    rng = data[1]
    original_date = data[2]
    start_date = original_date
    end_date = original_date
    rrule_str = dt["rrule"]
    if rng == "D":
        start_date = start_date.replace(hour=0, minute=0, second=0, microsecond=0)
        end_date = original_date.replace(day=original_date.day+1,hour=0,minute=0,second=0)
        date_iso_string = f'{end_date.isoformat().replace("-", "").replace(":", "")}Z'
        rrule_str = re.sub(r"COUNT=\d*", f'UNTIL={date_iso_string}', rrule_str)
    elif rng == "W":
        start_date = original_date - timedelta(days=original_date.weekday())
        end_date = start_date + timedelta(weeks=1)
        date_iso_string = f'{end_date.isoformat().replace("-", "").replace(":", "")}Z'
        rrule_str = re.sub(r"COUNT=\d*", f'UNTIL={date_iso_string}', rrule_str)
    elif rng == "M":
        start_date = original_date.replace(day=1)
        end_date = start_date.replace(month=original_date.month+1)
        date_iso_string = f'{end_date.isoformat().replace("-", "").replace(":", "")}Z'
        rrule_str = re.sub(r"COUNT=\d*", f'UNTIL={date_iso_string}', rrule_str)
    elif rng == "Y":
        start_date = original_date.replace(day=1,month=1)
        end_date = start_date.replace(year=original_date.year+1)
        date_iso_string = f'{end_date.isoformat().replace("-", "").replace(":", "")}Z'
        rrule_str = re.sub(r"COUNT=\d*", f'UNTIL={date_iso_string}', rrule_str)
    final_list = []
    date_iso_string = f'{start_date.isoformat().replace("-", "").replace(":", "")}Z'
    rrule_str = f"DTSTART:{date_iso_string}\n"+rrule_str
    # print(rrule_str)
    rrule_dt_list = list(rrule.rrulestr(rrule_str,dtstart=start_date))
    for i in rrule_dt_list:
        final_list.append(
            {
                **dt,
                "meetingDate": i.isoformat()
            }
        )
    return final_list
    

def get_meeting_data_by_range(date,rg=None):
    # rg = D for Date, M for month, Y for year
    # date = date of choice
    unsorted_list = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_data = {executor.submit(make_list_from_rrule,[data,rg,date]):data for data in meeting_list}
        for future in as_completed(future_to_data):
            data = future_to_data[future]
            final_list = future.result()
            unsorted_list.extend(final_list)
    return sort_meeting_by_date(unsorted_list)


            
    

# Meetings Today

In [113]:
week_data = get_meeting_data_by_range(datetime.now(), 'D')
for i in week_data:
    print(f"{i['meetingDate']} - {i['meetingTime']} - {i['meetingType']}")


2022-06-15T00:00:00+00:00 - 07:06 AM - Project interview season.
2022-06-15T00:00:00+00:00 - 22:06 PM - Whom nor according star story until group.
2022-06-15T00:00:00+00:00 - 09:15 AM - I tell offer team world their sort.
2022-06-15T00:00:00+00:00 - 01:33 AM - Can strategy various.
2022-06-15T00:00:00+00:00 - 15:08 PM - Human program buy most.
2022-06-15T00:00:00+00:00 - 11:31 AM - Attention lose name see.
2022-06-15T00:00:00+00:00 - 18:11 PM - Like senior senior teach north.
2022-06-15T00:00:00+00:00 - 02:03 AM - Blood between station table.
2022-06-15T00:00:00+00:00 - 15:21 PM - Day fly article garden represent threat.
2022-06-15T00:00:00+00:00 - 18:40 PM - Hospital politics maybe million individual.
2022-06-15T00:00:00+00:00 - 22:56 PM - Guess eye across fact scene order behavior officer.
2022-06-15T00:00:00+00:00 - 17:12 PM - Prepare different every radio single end myself car.
2022-06-15T00:00:00+00:00 - 15:00 PM - Reality necessary protect today race police letter.
2022-06-15T00:

# Meeting This Week On June

In [114]:
week_data = get_meeting_data_by_range(datetime.now(),'W')
for i in week_data:
    print(f"{i['meetingDate']} - {i['meetingTime']} - {i['meetingType']}")


2022-06-13T18:26:04+00:00 - 02:16 AM - Ago final small doctor.
2022-06-13T18:26:04+00:00 - 18:11 PM - Like senior senior teach north.
2022-06-13T18:26:04+00:00 - 22:06 PM - Whom nor according star story until group.
2022-06-13T18:26:04+00:00 - 07:06 AM - Project interview season.
2022-06-13T18:26:04+00:00 - 16:51 PM - Degree information show.
2022-06-13T18:26:04+00:00 - 22:56 PM - Guess eye across fact scene order behavior officer.
2022-06-13T18:26:04+00:00 - 17:28 PM - Wish on author ball sit last drop respond.
2022-06-13T18:26:04+00:00 - 14:56 PM - Few place message chance.
2022-06-13T18:26:04+00:00 - 02:03 AM - Blood between station table.
2022-06-13T18:26:04+00:00 - 06:28 AM - Purpose movie perhaps.
2022-06-13T18:26:04+00:00 - 15:21 PM - Day fly article garden represent threat.
2022-06-13T18:26:04+00:00 - 10:57 AM - Myself page last different make.
2022-06-13T18:26:04+00:00 - 02:49 AM - Cost huge key culture reflect.
2022-06-13T18:26:04+00:00 - 01:36 AM - True teach able physical p

# Meeting On Month Of June


In [115]:
june_data = get_meeting_data_by_range(datetime.now(),'M')
# regex pattern for finding number between decimal 50 to decimal 2000

 

In [116]:

for i in june_data:
    print(f"{i['meetingDate']} - {i['meetingTime']} - {i['meetingType']}")

2022-06-01T18:26:04+00:00 - 09:15 AM - I tell offer team world their sort.
2022-06-01T18:26:04+00:00 - 10:57 AM - Myself page last different make.
2022-06-01T18:26:04+00:00 - 02:03 AM - Blood between station table.
2022-06-01T18:26:04+00:00 - 17:12 PM - Prepare different every radio single end myself car.
2022-06-01T18:26:04+00:00 - 16:51 PM - Degree information show.
2022-06-01T18:26:04+00:00 - 17:28 PM - Wish on author ball sit last drop respond.
2022-06-01T18:26:04+00:00 - 18:11 PM - Like senior senior teach north.
2022-06-01T18:26:04+00:00 - 03:36 AM - Hold walk rather action.
2022-06-01T18:26:04+00:00 - 01:36 AM - True teach able physical past fly.
2022-06-01T18:26:04+00:00 - 22:56 PM - Guess eye across fact scene order behavior officer.
2022-06-01T18:26:04+00:00 - 15:08 PM - Human program buy most.
2022-06-01T18:26:04+00:00 - 07:06 AM - Project interview season.
2022-06-01T18:26:04+00:00 - 11:31 AM - Attention lose name see.
2022-06-01T18:26:04+00:00 - 18:30 PM - Like process cul